In [1]:
from __future__ import print_function, division
%pylab inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from bokeh.plotting import *

from bokeh.io import output_notebook
output_notebook()
from collections import Counter
from operator import itemgetter

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [2]:
df = pd.read_pickle('coursera_clf.df')

In [3]:
print(df.columns)
df.head(2)

Index([u'idx', u'jdx', u'vec-diff', u'cos-sim', u'euc-dist', u'man-dist',
       u'bin_diff', u'max_diff', u'mean_diff', u'median_diff', u'v1_max',
       u'v2_max', u'v1_min', u'v2_min', u'v1_mean', u'v2_mean', u'v1_median',
       u'v2_median', u'v1_count', u'v2_count', u'v1', u'v2', u'c1', u'c2',
       u'c_id', u'prereq_id'],
      dtype='object')


,idx,jdx,vec-diff,cos-sim,euc-dist,man-dist,bin_diff,max_diff,mean_diff,median_diff,...,v1_median,v2_median,v1_count,v2_count,v1,v2,c1,c2,c_id,prereq_id
0,0,0,0.000000,1.0,0.000000,0.000000,0,0.000000,0.000000,0.0,...,0.0,0.0,13,13,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Reproducible Research,Reproducible Research,-4ffSHNYEeWIfhKr_WcYsQ,-4ffSHNYEeWIfhKr_WcYsQ
1,0,1,1.010404,0.0,1.414214,7.161435,20,0.434719,0.000201,0.0,...,0.0,0.0,13,20,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Reproducible Research,Machine Learning: Clustering & Retrieval,-4ffSHNYEeWIfhKr_WcYsQ,-N44X0IJEeWpogr5ZO8qxQ


In [4]:
df = df[df['idx'] != df['jdx']]

In [5]:
features = ['vec-diff', 'cos-sim', 'euc-dist', 'man-dist', 'bin_diff',
            'max_diff', 'mean_diff', 'median_diff',
            'v1_max', 'v2_max', 'v1_min', 'v2_min', 'v1_mean', 'v2_mean', 'v1_median', 
            'v2_median', 'v1_count', 'v2_count']
print(features)

print('unused:', set(df.columns) - set(features))

['vec-diff', 'cos-sim', 'euc-dist', 'man-dist', 'bin_diff', 'max_diff', 'mean_diff', 'median_diff', 'v1_max', 'v2_max', 'v1_min', 'v2_min', 'v1_mean', 'v2_mean', 'v1_median', 'v2_median', 'v1_count', 'v2_count']
unused: set(['c_id', 'idx', 'prereq_id', 'v1', 'v2', 'jdx', 'c2', 'c1'])


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedShuffleSplit, StratifiedKFold
import random
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score
import copy
import joblib

In [13]:
clf = joblib.load('random_forest.clf')

In [14]:
true_class_idx = list(filter(lambda x: x[1] > 0, enumerate(clf.classes_)))[0][0]
scaler = StandardScaler()
df['pred'] = clf.predict_proba(scaler.fit_transform(df[features].fillna(0)))[:, true_class_idx]

In [15]:
df.sort_values(by='pred', ascending=False).head(100)[['c1', 'c2', 'pred']]

,c1,c2,pred
196,Reproducible Research,Customer Analytics,0.426554
111223,Core Concepts in Data Analysis,Customer Analytics,0.426554
85162,Getting and Cleaning Data,Customer Analytics,0.426554
88375,Python Data Structures,Customer Analytics,0.426554
118006,Healthcare Data Visualization,Customer Analytics,0.426554
113008,Discrete Optimization,Customer Analytics,0.426554
115150,Linear and Discrete Optimization,Customer Analytics,0.426554
6979,Tutorial on Neural Network Optimization Proble...,Customer Analytics,0.426554
121576,Social Network Analysis,Customer Analytics,0.426554
6265,New Developments in the Theory of Clustering -...,Customer Analytics,0.426554
